In [79]:
import numpy as np
import pandas as pd

# config
tile_x = 0 #0, 1, 2, 3
tile_y = 0 #0, 1, 2, 3
kernel = 7 #0, 1, ..., 31
IFM_dim = 224

data_in = pd.read_csv('../data/input.txt', sep=" ", header=None, dtype=np.int32)

in_np_ch0 = np.zeros((226,226), np.int32)
in_np_ch1 = np.zeros((226,226), np.int32)
in_np_ch2 = np.zeros((226,226), np.int32)

in_np_ch0[1:225,1:225] = data_in[:224].to_numpy()
in_np_ch1[1:225,1:225] = data_in[224:448].to_numpy()
in_np_ch2[1:225,1:225] = data_in[448:672].to_numpy()

t_idx_y = tile_y*60
t_idx_x = tile_x*60

if(t_idx_x+61 > IFM_dim+2):
    t_idx_x_r = 226
else:
    t_idx_x_r = t_idx_x+61

if(t_idx_y+61 > IFM_dim+2):
    t_idx_y_b = 226
else:
    t_idx_y_b = t_idx_y+61

tile_ch0 = in_np_ch0[t_idx_y:t_idx_y+61, t_idx_x:t_idx_x+61]
tile_ch1 = in_np_ch1[t_idx_y:t_idx_y+61, t_idx_x:t_idx_x+61]
tile_ch2 = in_np_ch2[t_idx_y:t_idx_y+61, t_idx_x:t_idx_x+61]

print(tile_ch0.shape)
tile_ch0

(61, 61)


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,  83,  83, ...,  78,  79,  79],
       [  0,  81,  81, ...,  82,  80,  82],
       ...,
       [  0, 103, 107, ...,  69,  67,  66],
       [  0,  88,  94, ...,  71,  67,  65],
       [  0,  72,  79, ...,  82,  77,  70]], dtype=int32)

In [80]:
data_in = pd.read_csv('../data/weight.txt', sep=" ", header=None, dtype=np.int32)
in_np = data_in.to_numpy()

w_idx = kernel*9
weight_ch0 = in_np[w_idx:w_idx+3]
weight_ch1 = in_np[w_idx+3:w_idx+6]
weight_ch2 = in_np[w_idx+6:w_idx+9]

weight_ch0

array([[ 1, -1, -6],
       [ 0,  4, 11],
       [-1,  7, 17]], dtype=int32)

In [81]:
data_in = pd.read_csv('../data/Q_Bias.txt', sep=" ", header=None, dtype=np.int32)
in_np = data_in.to_numpy()
in_np = np.squeeze(in_np)

bias = in_np[kernel]

bias

11732

In [82]:
stride = 2
# bias = np.int16(-1589)
p_sum = np.int64(0)

OFM = np.zeros((30,30), np.int32)

OFM_y_idx = 0
for y in range(3, np.size(tile_ch0, 1)+1, stride) :
    OFM_x_idx = 0

    for x in range(3, np.size(tile_ch0, 1)+1, stride):
        p_sum = 0

        window = tile_ch0[y-3:y,x-3:x]
        for c in range(3):
            for r in range(3):
                p_sum += weight_ch0[c,r] * window[c,r]

        # window = tile_ch1[y-3:y,x-3:x]
        # for c in range(3):
        #     for r in range(3):
        #         p_sum += weight_ch1[c,r] * window[c,r]

        # window = tile_ch2[y-3:y,x-3:x]
        # for c in range(3):
        #     for r in range(3):
        #         p_sum += weight_ch2[c,r] * window[c,r]

        if(p_sum + bias >= 8388608):
            OFM[OFM_y_idx,OFM_x_idx] = 8388608
        elif(p_sum + bias <= -8388608):
            OFM[OFM_y_idx,OFM_x_idx] = -8388608
        else :
            OFM[OFM_y_idx,OFM_x_idx] = p_sum + bias
            
        OFM_x_idx += 1

    OFM_y_idx += 1

In [83]:

# f_name = "OFM_x" + str(tile_x) + "_y" + str(tile_y) + "_k" + str(kernel) + ".hex"
f_name = "OFM.hex"

if(kernel != 0):
    fin = open(f_name, 'r')
    content = fin.read()
    content_list = content.splitlines()
    fin.close()
else:
    open(f_name, 'w').close()
    content_list = []

r = 0
for row in range(OFM_y_idx):
    # fout.write("\t0x")
    for col in range( OFM_x_idx ):
        # fout.write( '{:06x}'.format( OFM[row,col] & ((1 << 16) - 1) )  )
        # fout.write(' ')
        # Johnson Liu
        if(kernel == 0):
            content_list.append('{:04x}'.format( OFM[row,col] & ((1 << 16) - 1)))
        else:
            print('{:04x}'.format( OFM[row,col] & ((1 << 16) - 1)))
            content_list[r] = '{}{:04x}'.format(content_list[r], OFM[row,col] & ((1 << 16) - 1))
        r += 1
    # if(row < np.size(OFM, 0)-1):
    #     fout.write("\n")
fout = open(f_name, 'w')
for l in range(len(content_list)):
    fout.write(str(content_list[l]) + '\n')
fout.close()

3a49
3a28
3a7d
3a84
3a36
3a3a
3af3
3af9
3a67
3a7b
3a7a
3a8e
3a46
3ad9
3ab8
3a0f
3a20
39d2
395a
39a6
3aba
3a8f
3ac6
3aa6
3ac1
397a
39c2
3a5e
39d9
39c5
382b
37d2
37c8
37b4
369b
36bb
372c
376d
376d
37ae
3844
383e
377a
386f
388c
3840
37d7
379b
3788
3777
3792
3759
3802
38d5
3912
3861
379e
37b8
387d
38a4
379a
37f0
38bd
3976
377a
3612
369f
3719
376f
3759
372b
3715
36cb
37ab
37cf
37ca
3851
388b
3868
380c
37ca
3796
37b1
3834
38cd
38be
37df
37a3
3823
387a
3819
38f2
3a3e
3b94
39a6
38bf
394c
39d6
3a3b
3a7a
3a2d
3947
38d8
3867
371f
3784
37f5
37ef
381e
388b
3849
383d
3854
387c
38bf
3804
37d1
3804
3894
38ad
3a11
3a4f
3a9f
3af4
3ad0
3ac1
3a8b
3a6e
3ab6
3b41
3b0f
3ad4
3bae
3a2a
38d2
38e6
399b
39ee
39bd
3933
385a
38b4
38ca
3921
38e1
3835
377f
376c
3830
390b
3872
38b9
38bf
38f4
392b
39c8
39ca
3992
39f8
3a0b
39fb
398d
3a61
3a65
39dc
3a83
3a36
3954
38e4
3976
3b08
3b4c
3ae6
3ad6
3ab2
3a64
39bb
3a8f
3ac1
3ac7
3731
3838
399b
399e
38e7
38a0
38b5
38c5
38a1
38bb
380a
3737
380a
38fb
3835
3828
3867
37f3
3801
381f
